# Install and import libraries



In [ ]:
!pip install pytesseract transformers datasets rouge-score nltk tensorboard py7zr --upgrade
!pip install music21
!pip install note_seq
!pip install torch==1.8.0+cu111 -f https://download.pytorch.org/whl/cu111/torch_stable.html
!pip install evaluate

In [ ]:
!sudo pip install pyfluidsynth

sudo: pip: command not found


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from music21 import *
import itertools
import numpy as np
import pandas as pd
import random
import json
import os
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2Config, DataCollatorWithPadding, pipeline, TrainingArguments, Trainer, EarlyStoppingCallback
import torch
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
import evaluate
import nltk
from nltk.tokenize import sent_tokenize
from huggingface_hub import HfFolder

2023-05-10 23:46:21.171165: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-10 23:46:22.066309: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/jupyter/TensorRT-8.0.3.4/lib:/usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-05-10 23:46:22.066431: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/jupyter/Ten

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import WhitespaceSplit
from tokenizers.trainers import WordLevelTrainer

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Preprocessing MIDI data

### 1. Translating MIDI into data

The following 5 methods are modified from https://github.com/AI-Guru/MMM-JSB

In [ ]:
def events_to_events_data(events):
    '''
    Encode note events (on, off, time delta) into corresponding event data
    '''
    events = sorted(events, key=lambda event: event[2])

    events_data = []
    for event_index, event, event_next in zip(range(len(events)), events, events[1:] + [None]):
        if event_index == 0 and event[2] != 0.0:
            # if the first event is time delta
            event_data = {
                "type": "TIME_DELTA",
                "delta": event[2]
            }
            events_data += [event_data]
        # note on or note off event
        event_data = {
            "type": event[0],
            "pitch": event[1]
        }
        events_data += [event_data]

        if event_next is None:
            continue

        # calculate time delta between this and next note
        delta = event_next[2] - event[2]
        assert delta >= 0, events
        if delta != 0.0:
            event_data = {
                "type": "TIME_DELTA",
                "delta": delta
            }
            events_data += [event_data]

    return events_data

In [ ]:
def preprocess_measure(measure):
    '''
    Process a measure with events
    '''
    bar_data = {}
    bar_data["events"] = []

    events = []
    for note in measure.recurse(classFilter=("Note")):
        #print("        NOTE", note.pitch.midi, note.offset, note.duration.quarterLength)
        events += [("NOTE_ON", note.pitch.midi, 4 * note.offset)]
        events += [("NOTE_OFF", note.pitch.midi, 4 * note.offset + 4 * note.duration.quarterLength)]

    bar_data["events"] = events_to_events_data(events)
    return bar_data

In [ ]:
def preprocess_part(part, part_index):
    '''
    Process every measure in each track/voice
    '''
    track_data = {}
    # local mapping for track names
    name = {'Soprano': 1, 'Alto': 2, 'Tenor': 3, 'Bass': 4}
    track_data["name"] = part.partName
    track_data["number"] = name[part.partName]
    track_data["bars"] = []

    for measure_index in range(1000):
        measure = part.measure(measure_index)
        if measure is None:
            break
        bar_data = preprocess_measure(measure)
        track_data["bars"] += [bar_data]
    return track_data

In [1]:
def preprocess_one_song(song):
    '''
    Process every track/voice in each song
    '''
    # only take the following 4 parts from the song
    parts = ['Soprano', 'Alto', 'Tenor', 'Bass']
    # skip everything that has multiple meters
    meters = [meter.ratioString for meter in song.recurse().getElementsByClass(meter.TimeSignature)]
    meters = list(set(meters))
    if len(meters) != 1:
        return None
    #elif meters[0] != "4/4":
    #    return None

    song_data = {}
    song_data["title"] = song.metadata.title
    song_data["number"] = song.metadata.number
    song_data["tracks"] = []
    for part_index, part in enumerate(song.parts):
        if part.partName in parts:
            track_data = preprocess_part(part, part_index)
            if (len(track_data["bars"]) > 0):
                song_data["tracks"] += [track_data]

    # if not all 4 parts exist
    if (len(song_data['tracks']) != 4):
        return None
    
    # randomly shuffle the tracks to train model later
    random.shuffle(song_data['tracks'])
    return song_data

In [ ]:
def preprocess_multiple_songs(songs):
    '''
    Process all songs in the corpus
    '''
    data = []
    for song in songs:
        song_data = preprocess_one_song(song)
        # if song data is valid
        if song_data is not None:
            data += [song_data]
    return data

Create song list from the music21 library corpus, using only songs with 4/4 time measure

In [ ]:
songs = corpus.search('4/4')
songList = []
for i in range(len(songs)):
    songList += [songs[i].parse()]

Process the song list

In [ ]:
songs_data = preprocess_multiple_songs(songList)

### 2. Create text input from music data

Test output from previous section

In [ ]:
songs_data[1]

{'title': 'bwv101.7.mxl',
 'number': None,
 'tracks': [{'name': 'Bass',
   'number': 4,
   'bars': [{'events': [{'type': 'NOTE_ON', 'pitch': 50},
      {'type': 'TIME_DELTA', 'delta': 4.0},
      {'type': 'NOTE_OFF', 'pitch': 50}]},
    {'events': [{'type': 'NOTE_ON', 'pitch': 49},
      {'type': 'TIME_DELTA', 'delta': 4.0},
      {'type': 'NOTE_OFF', 'pitch': 49},
      {'type': 'NOTE_ON', 'pitch': 50},
      {'type': 'TIME_DELTA', 'delta': 4.0},
      {'type': 'NOTE_OFF', 'pitch': 50},
      {'type': 'NOTE_ON', 'pitch': 58},
      {'type': 'TIME_DELTA', 'delta': 4.0},
      {'type': 'NOTE_OFF', 'pitch': 58},
      {'type': 'NOTE_ON', 'pitch': 53},
      {'type': 'TIME_DELTA', 'delta': 4.0},
      {'type': 'NOTE_OFF', 'pitch': 53}]},
    {'events': [{'type': 'NOTE_ON', 'pitch': 55},
      {'type': 'TIME_DELTA', 'delta': 4.0},
      {'type': 'NOTE_OFF', 'pitch': 55},
      {'type': 'NOTE_ON', 'pitch': 57},
      {'type': 'TIME_DELTA', 'delta': 4.0},
      {'type': 'NOTE_OFF', 'pitch': 

In [ ]:
type(songs_data[0]['tracks'])

In [ ]:
def encode_one_song(song_data):
    '''
    Encode a song from song data into tokens
    '''

    # Calculate the number of samples, each sample has 4 measures of music data
    num_bar = len(song_data['tracks'][0]['bars'])
    for track in song_data['tracks']:
        num_bar = min(num_bar, len(track['bars']))
    samples = [''] * (num_bar // 4)

    # Create token sequence
    for track in song_data['tracks']:
        # Each sequence starts with the instrument number
        sequence = 'INST=' + str(track['number']) + ' '
        for i in range(num_bar):
            # If we are processing a new sample
            if i % 4 == 0 and i > 0:
                # Add the current sequence to the sample list
                samples[i // 4 - 1] += sequence
                # Initiate new sequence
                sequence = 'INST=' + str(track['number']) + ' '
            bar = track['bars'][i]
            sequence += 'BAR_START '
            # Add music events to sequence
            for event in bar['events']:
                sequence += event['type'] + "="
                # If the event type is time delta
                if event['type'] == 'TIME_DELTA':
                    sequence += str(event['delta']) + " "
                # If the event type is note on or note off
                else:
                    sequence += str(event['pitch']) + " "
            sequence += 'BAR_END '
    # Get rid of trailing whitespace
    return [sample[:-1] for sample in samples]

In [ ]:
def encode_all_songs(songs_data):
    '''
    Encode all songs in song list 
    '''
    target = []
    for song in songs_data:
        target += encode_one_song(song)
    return target

Encode all songs in song list and randomly shuffle them

In [ ]:
encoded_data = encode_all_songs(songs_data)
random.shuffle(encoded_data)

# Create Dataset and Tokenizer

### 1. Create Dataset

In [ ]:
def create_data(encoded_data):
    '''
    Create token sequence that has the last track in the data masked
    '''

    # This will be returned
    input_data = []

    # Replace the last track with fill sequence
    fill_sequence = 'BAR_START [MASK] '

    for sample in encoded_data:
        # Get position of tracks
        indices = [0] * 5
        start_index = 0
        for i in range(4):
            indices[i] = sample.find('INST', start_index)
            start_index = indices[i] + 1
        indices[4] = len(sample)
        # Fill in the last track
        chosen = 3
        # Create input and output
        input_sequence = sample[:indices[chosen] + 7] + fill_sequence + sample[indices[chosen + 1]:]
        if (chosen == 3):
            # Remove trailing whitespace
            input_sequence = input_sequence[:-1]
        input_data += [input_sequence]
    return input_data

Split data into train and test (validation) set

In [ ]:
split_index = (int)(0.8 * len(encoded_data))
# Label is the same as input for training data
input_data_train = encoded_data[:split_index]
target_data_train = encoded_data[:split_index]
input_data_valid = create_data(encoded_data[split_index:])
target_data_valid = encoded_data[split_index:]

Verify the functions work

In [ ]:
input_data_valid[1]

'INST=3 BAR_START NOTE_ON=58 TIME_DELTA=2.0 NOTE_OFF=58 NOTE_ON=60 TIME_DELTA=2.0 NOTE_OFF=60 BAR_END BAR_START NOTE_ON=62 TIME_DELTA=4.0 NOTE_OFF=62 NOTE_ON=58 TIME_DELTA=4.0 NOTE_OFF=58 NOTE_ON=57 TIME_DELTA=2.0 NOTE_OFF=57 NOTE_ON=55 TIME_DELTA=2.0 NOTE_OFF=55 NOTE_ON=53 TIME_DELTA=4.0 NOTE_OFF=53 BAR_END BAR_START NOTE_ON=58 TIME_DELTA=4.0 NOTE_OFF=58 NOTE_ON=58 TIME_DELTA=4.0 NOTE_OFF=58 NOTE_ON=60 TIME_DELTA=6.0 NOTE_OFF=60 NOTE_ON=58 TIME_DELTA=2.0 NOTE_OFF=58 BAR_END BAR_START NOTE_ON=57 TIME_DELTA=4.0 NOTE_OFF=57 NOTE_ON=57 TIME_DELTA=1.0 NOTE_OFF=57 NOTE_ON=58 TIME_DELTA=1.0 NOTE_OFF=58 NOTE_ON=60 TIME_DELTA=4.0 NOTE_OFF=60 NOTE_ON=58 TIME_DELTA=2.0 NOTE_OFF=58 NOTE_ON=62 TIME_DELTA=4.0 NOTE_OFF=62 BAR_END INST=4 BAR_START NOTE_ON=43 TIME_DELTA=2.0 NOTE_OFF=43 NOTE_ON=45 TIME_DELTA=2.0 NOTE_OFF=45 BAR_END BAR_START NOTE_ON=46 TIME_DELTA=4.0 NOTE_OFF=46 NOTE_ON=43 TIME_DELTA=4.0 NOTE_OFF=43 NOTE_ON=45 TIME_DELTA=4.0 NOTE_OFF=45 NOTE_ON=46 TIME_DELTA=4.0 NOTE_OFF=46 BAR_END BAR

In [ ]:
target_data_train[0]

'INST=2 BAR_START NOTE_ON=59 TIME_DELTA=4.0 NOTE_OFF=59 BAR_END BAR_START NOTE_ON=64 TIME_DELTA=4.0 NOTE_OFF=64 NOTE_ON=66 TIME_DELTA=4.0 NOTE_OFF=66 NOTE_ON=67 TIME_DELTA=4.0 NOTE_OFF=67 NOTE_ON=66 TIME_DELTA=4.0 NOTE_OFF=66 BAR_END BAR_START NOTE_ON=71 TIME_DELTA=6.0 NOTE_OFF=71 NOTE_ON=70 TIME_DELTA=2.0 NOTE_OFF=70 NOTE_ON=66 TIME_DELTA=4.0 NOTE_OFF=66 NOTE_ON=66 TIME_DELTA=4.0 NOTE_OFF=66 BAR_END BAR_START NOTE_ON=66 TIME_DELTA=2.0 NOTE_OFF=66 NOTE_ON=67 TIME_DELTA=2.0 NOTE_OFF=67 NOTE_ON=67 TIME_DELTA=2.0 NOTE_OFF=67 NOTE_ON=66 TIME_DELTA=2.0 NOTE_OFF=66 NOTE_ON=66 TIME_DELTA=2.0 NOTE_OFF=66 NOTE_ON=64 TIME_DELTA=2.0 NOTE_OFF=64 NOTE_ON=64 TIME_DELTA=2.0 NOTE_OFF=64 NOTE_ON=63 TIME_DELTA=2.0 NOTE_OFF=63 BAR_END INST=4 BAR_START NOTE_ON=52 TIME_DELTA=4.0 NOTE_OFF=52 BAR_END BAR_START NOTE_ON=52 TIME_DELTA=4.0 NOTE_OFF=52 NOTE_ON=50 TIME_DELTA=4.0 NOTE_OFF=50 NOTE_ON=55 TIME_DELTA=4.0 NOTE_OFF=55 NOTE_ON=50 TIME_DELTA=2.0 NOTE_OFF=50 NOTE_ON=49 TIME_DELTA=2.0 NOTE_OFF=49 BAR_END BAR

Save token data into text files so we don't have to go back and preprocess again next time

In [ ]:
def token_to_txt_file(token_sequences, path):
    with open(path, "w") as file:
        for token_sequence in token_sequences:
            print(token_sequence, file=file)
            
def txt_file_to_token(path):
    fin = open(path, "r")
    lines = fin.readlines()
    token_sequences = []
    for line in lines:
        token_sequences.append(line.strip())
    return token_sequences

In [ ]:
token_to_txt_file(input_data_train, "input_data_train.txt")
token_to_txt_file(target_data_train, "target_data_train.txt")
token_to_txt_file(input_data_valid, "input_data_valid.txt")
token_to_txt_file(target_data_valid, "target_data_valid.txt")

### 2. Create tokenizer

The following function is modified from https://github.com/AI-Guru/MMM-JSB/

In [ ]:
def create_tokenizer(files):
    tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
    tokenizer.pre_tokenizer = WhitespaceSplit()
    tokens = ["[UNK]", "[PAD]"]
    trainer = WordLevelTrainer(special_tokens=tokens)
    tokenizer.train(files=files, trainer=trainer)
    return tokenizer

Create tokenizer and add the MASK token, then save tokenizer to a json file to not have to preprocess next time.

In [ ]:
tokenizer = create_tokenizer(files=["input_data_train.txt", "target_data_train.txt",
                                    "input_data_valid.txt", "target_data_valid.txt"])
tokenizer.mask_token = "<MASK>"
tokenizer.save("tokenizer.json")

Check the vocab list of tokenizer

In [ ]:
tokenizer.get_vocab()

{'NOTE_OFF=38': 96,
 'NOTE_OFF=72': 35,
 'NOTE_OFF=65': 28,
 'TIME_DELTA=4.0': 2,
 'NOTE_ON=74': 33,
 'NOTE_OFF=43': 72,
 'NOTE_OFF=51': 74,
 'NOTE_ON=79': 88,
 'NOTE_ON=75': 79,
 'NOTE_ON=73': 59,
 'NOTE_ON=43': 73,
 'NOTE_ON=50': 27,
 'INST=1': 50,
 'NOTE_OFF=66': 20,
 'NOTE_ON=53': 43,
 'NOTE_OFF=36': 100,
 'NOTE_OFF=81': 104,
 'NOTE_ON=60': 17,
 '[UNK]': 0,
 'NOTE_ON=62': 7,
 'NOTE_OFF=57': 10,
 'NOTE_OFF=50': 26,
 'NOTE_ON=63': 61,
 'TIME_DELTA=12.0': 64,
 'NOTE_OFF=75': 78,
 'NOTE_ON=52': 31,
 'NOTE_OFF=79': 87,
 'NOTE_ON=66': 21,
 'NOTE_ON=51': 75,
 'NOTE_ON=56': 66,
 'NOTE_OFF=63': 60,
 'TIME_DELTA=2.0': 3,
 'NOTE_ON=70': 55,
 'NOTE_OFF=40': 94,
 'TIME_DELTA=3.0': 89,
 'NOTE_OFF=56': 65,
 'NOTE_ON=69': 15,
 'NOTE_ON=49': 71,
 'NOTE_OFF=70': 54,
 'NOTE_OFF=46': 76,
 'NOTE_OFF=42': 90,
 'NOTE_OFF=52': 30,
 'NOTE_ON=38': 97,
 'NOTE_ON=44': 93,
 'NOTE_OFF=59': 18,
 'NOTE_OFF=55': 22,
 'NOTE_OFF=47': 56,
 'INST=4': 53,
 'NOTE_ON=39': 103,
 'NOTE_ON=81': 105,
 'NOTE_OFF=44': 92,
 'NO

In [ ]:
#tokenizer = Tokenizer.from_file("tokenizer.json")

Create a fast tokenizer from the tokenizer

In [ ]:
fast_tokenizer = PreTrainedTokenizerFast(tokenizer_file="tokenizer.json")
fast_tokenizer.add_special_tokens({'pad_token': '[PAD]'})
fast_tokenizer.add_special_tokens({'mask_token': '[MASK]'})

0

In [ ]:
fast_tokenizer.get_vocab()

{'NOTE_ON=49': 71,
 'NOTE_OFF=76': 67,
 'NOTE_OFF=75': 78,
 'INST=1': 50,
 'NOTE_OFF=47': 56,
 'NOTE_OFF=54': 37,
 'NOTE_ON=48': 47,
 'NOTE_OFF=42': 90,
 'NOTE_OFF=48': 46,
 'NOTE_ON=42': 91,
 'NOTE_OFF=51': 74,
 'TIME_DELTA=2.0': 3,
 'NOTE_OFF=73': 58,
 'NOTE_OFF=67': 12,
 'TIME_DELTA=4.0': 2,
 'NOTE_OFF=62': 6,
 'NOTE_OFF=52': 30,
 'NOTE_OFF=55': 22,
 'NOTE_OFF=77': 80,
 'NOTE_OFF=78': 82,
 'NOTE_ON=45': 49,
 'NOTE_ON=50': 27,
 'NOTE_ON=78': 83,
 'NOTE_ON=69': 15,
 'NOTE_OFF=63': 60,
 'NOTE_ON=36': 101,
 'NOTE_ON=46': 77,
 'NOTE_OFF=79': 87,
 'NOTE_OFF=45': 48,
 'NOTE_ON=81': 105,
 'NOTE_OFF=44': 92,
 'NOTE_ON=71': 25,
 'NOTE_OFF=64': 8,
 'INST=3': 52,
 'NOTE_ON=55': 23,
 'NOTE_ON=61': 45,
 'NOTE_ON=44': 93,
 'NOTE_ON=39': 103,
 'NOTE_OFF=59': 18,
 'NOTE_ON=38': 97,
 '[PAD]': 1,
 'INST=4': 53,
 'NOTE_OFF=60': 16,
 'NOTE_ON=65': 29,
 'NOTE_OFF=58': 40,
 'NOTE_ON=63': 61,
 'NOTE_OFF=56': 65,
 'NOTE_OFF=81': 104,
 'NOTE_OFF=68': 62,
 'TIME_DELTA=6.0': 69,
 'NOTE_ON=72': 36,
 'TIME_DELTA

Check that the MASK token is in the vocabulary of the tokenizer

In [ ]:
fast_tokenizer.mask_token_id

86

In [ ]:
#encoded_data = txt_file_to_token("input_data_train.txt") + txt_file_to_token("target_data_valid.txt")

Tokenize the data with the fast tokenizer

In [ ]:
tokenized_data = list(map(lambda x: fast_tokenizer.encode(x), encoded_data))

Calculate the max source length

In [ ]:
max_source_length = max([len(x) for x in tokenized_data])
max_source_length

339

Make sure that the PAD token is in the vocabulary

In [ ]:
fast_tokenizer.encode("[PAD]")

[1]

### 3. Create tokenized dataset

The following function is modified from https://github.com/AI-Guru/MMM-JSB/

In [ ]:
class TokenSequenceDataset(Dataset):
    def __init__(self, tokenizer, input_dataset_path, target_dataset_path, block_size):
        pad_token_id = tokenizer.encode("[PAD]")[0]
        unk_token_id = tokenizer.encode("[UNK]")[0]
        input_lines = open(input_dataset_path, "r").readlines()
        target_lines = open(target_dataset_path, "r").readlines()    
            
        self.data = []
        unknown_tokens_set = []
        unknown_tokens = []
        
        for index in range(len(input_lines)):
            #Get current line
            input_line = input_lines[index]
            target_line = target_lines[index]
            
            #Skip empty lines.
            input_line = input_line.strip()
            target_line = target_line.strip()
            if not input_line or not target_line:
                continue

            # Encode the line.
            encoded_input_line = tokenizer.encode(input_line)
            encoded_target_line = tokenizer.encode(target_line)
            
            # Create a warning about unknown tokens. And then skip the line.
            if unk_token_id in encoded_input_line:
                print("unknown token")
                index = encoded_input_line.index(unk_token_id)
                token = tokenizer.decode(encoded_input_line[index])
                token = input_line.split()[index]
                if token not in unknown_tokens_set:
                    unknown_tokens_set += [token]
                #logger.warning(f"Skipping line because of unknown token {token}")
                unknown_tokens += [token]
                continue
            
            if unk_token_id in encoded_target_line:
                print("unknown token")
                index = encoded_target_line.index(unk_token_id)
                token = tokenizer.decode(encoded_target_line[index])
                token = target_line.split()[index]
                if token not in unknown_tokens_set:
                    unknown_tokens_set += [token]
                #logger.warning(f"Skipping line because of unknown token {token}")
                unknown_tokens += [token]
                continue
            
            # Skip sequence if it is too long.
            if len(encoded_input_line) > block_size or len(encoded_target_line) > block_size:
                continue
            
            # Pad and truncate.
            input_tensor = np.full((block_size,), pad_token_id, dtype=np.compat.long)
            input_tensor[:len(encoded_input_line)] = encoded_input_line
            target_tensor = np.full((block_size,), pad_token_id, dtype=np.compat.long)
            target_tensor[:len(encoded_target_line)] = encoded_target_line
            assert len(input_tensor) == block_size
            assert len(target_tensor) == block_size
            
            self.data.append({
                "input_ids": torch.tensor(input_tensor, dtype=torch.long),
                "attention_mask": [1] * block_size,
                "labels": torch.tensor(target_tensor, dtype=torch.long)
            })
            
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]

Create the train and test (validation) dataset

In [ ]:
dataset_train = TokenSequenceDataset(
            tokenizer=fast_tokenizer,
            input_dataset_path="input_data_train.txt",
            target_dataset_path="target_data_train.txt",
            block_size=max_source_length)

In [ ]:
dataset_valid = TokenSequenceDataset(
            tokenizer=fast_tokenizer,
            input_dataset_path="input_data_valid.txt",
            target_dataset_path="target_data_valid.txt",
            block_size=max_source_length)

# Create NLP Model 

The configuration of the model is modified from paper "MMM: Exploring Conditional Multi-Track Music Generation with the Transformer" (https://arxiv.org/abs/2008.06048)

In [ ]:
# Create the model.
model_config = GPT2Config(
    vocab_size=fast_tokenizer.vocab_size,
    pad_token_id=fast_tokenizer.convert_tokens_to_ids("[PAD]"),
    mask_token_id=fast_tokenizer.convert_tokens_to_ids("[MASK]"),
    n_head=8,
    n_layer=6,
    n_embd=512,
    n_positions=1024,
    n_ctx=1024,
    dropout=0.1,
    weight_decay=0.01,
)
model = GPT2LMHeadModel(model_config)

In [ ]:
data_collator = DataCollatorWithPadding(
            tokenizer=fast_tokenizer,
            padding="max_length",
            max_length=max_source_length)

In [ ]:
# local repository id
repository_id = "training"

training_args = TrainingArguments(
    output_dir=repository_id,
    overwrite_output_dir=True,
    evaluation_strategy="steps",
    num_train_epochs=5,
    learning_rate=5e-5,
    warmup_steps=100,                
    weight_decay=0.01,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    save_steps=1_000,
    save_total_limit=2,
    prediction_loss_only=False,
    logging_strategy="steps",
    logging_steps=100,
    logging_dir=f"{repository_id}/logs",
    load_best_model_at_end=True,
    save_strategy="steps",
    push_to_hub=False,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
    greater_is_better=False,
    metric_for_best_model="eval_loss",
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset_train,
    eval_dataset=dataset_valid,
    tokenizer=fast_tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

In [ ]:
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
100,2.258300,2.773087
200,1.446400,2.706568
300,1.218200,2.561710
400,1.012800,2.467211
500,0.927300,2.443300
600,0.855400,2.418327
700,0.811000,2.408300


TrainOutput(global_step=785, training_loss=1.1730323742909037, metrics={'train_runtime': 107.4995, 'train_samples_per_second': 29.116, 'train_steps_per_second': 7.302, 'total_flos': 120422922485760.0, 'train_loss': 1.1730323742909037, 'epoch': 5.0})

In [ ]:
trainer.save_model("best_model")

In [ ]:
fast_tokenizer.push_to_hub("my-model-gpt2", commit_message="tokenizer")
model_hub = model.push_to_hub("my-model-gpt2", tokenizer=fast_tokenizer, commit_message="My model GPT2")

pytorch_model.bin:   0%|          | 0.00/84.3M [00:00<?, ?B/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

# Generate results

In [ ]:
#generator = pipeline("text-generation", model="lam-le/my-model-gpt2", device=0)

In [ ]:
fast_tokenizer.get_vocab()

{'NOTE_ON=39': 103,
 '[PAD]': 1,
 'NOTE_ON=36': 101,
 'NOTE_ON=72': 36,
 'NOTE_ON=73': 57,
 'NOTE_ON=74': 33,
 'NOTE_OFF=42': 90,
 'NOTE_ON=40': 95,
 'NOTE_ON=65': 27,
 'INST=4': 55,
 'INST=2': 53,
 'NOTE_ON=68': 63,
 'NOTE_OFF=53': 40,
 'NOTE_ON=45': 51,
 'INST=3': 54,
 'NOTE_ON=80': 107,
 'NOTE_ON=71': 25,
 'NOTE_ON=56': 66,
 'NOTE_ON=78': 84,
 'NOTE_ON=63': 61,
 'NOTE_ON=59': 19,
 'NOTE_ON=42': 91,
 'NOTE_OFF=81': 104,
 'NOTE_OFF=62': 6,
 'NOTE_OFF=67': 12,
 'NOTE_ON=67': 13,
 'TIME_DELTA=1.0': 39,
 'NOTE_ON=58': 43,
 'NOTE_OFF=65': 26,
 'NOTE_OFF=40': 94,
 'TIME_DELTA=12.0': 64,
 'NOTE_ON=69': 15,
 'NOTE_OFF=76': 67,
 'NOTE_OFF=73': 56,
 'NOTE_ON=43': 73,
 'NOTE_OFF=48': 46,
 'NOTE_ON=62': 7,
 'NOTE_ON=54': 38,
 'NOTE_ON=46': 77,
 'TIME_DELTA=8.0': 34,
 'NOTE_ON=75': 80,
 'NOTE_ON=61': 45,
 'NOTE_OFF=66': 20,
 'NOTE_OFF=54': 37,
 'NOTE_OFF=70': 48,
 'NOTE_OFF=55': 22,
 '[MASK]': 78,
 'NOTE_OFF=41': 85,
 'NOTE_OFF=74': 32,
 'NOTE_ON=64': 9,
 'TIME_DELTA=3.0': 89,
 'NOTE_ON=66': 21,


In [ ]:
def generate(model, tokenizer, token_sequence):

    # Map token sequence to ids.
    input_ids = tokenizer.encode(token_sequence, add_special_tokens=True, return_tensors="pt")

    with torch.no_grad():
        output = model.generate(input_ids=input_ids, 
                                max_length=max_source_length, 
                                do_sample=True,
                                top_p=0.95,
                                top_k=50,)
        ''',
            #min_length=200,
            max_length=400,
            do_sample=True,
            top_p=0.95,
            top_k=50,
            num_return_sequences=1,
            #bos_token_id=tokenizer.token_to_id("PIECE_START"),
            #eos_token_id=tokenizer.token_to_id("PIECE_END"),
            #bad_words_ids=[[tokenizer.token_to_id("[PAD]")], [tokenizer.token_to_id("[MASK]")]]
        )'''

        generated_sequence = tokenizer.decode(output[0], skip_special_tokens=True)
        return generated_sequence

In [ ]:
input_data_valid = txt_file_to_token("input_data_valid.txt")
target_data_valid = txt_file_to_token("target_data_valid.txt")

In [ ]:
input_data_valid[0]

'INST=3 BAR_START NOTE_ON=57 TIME_DELTA=4.0 NOTE_OFF=57 NOTE_ON=57 TIME_DELTA=4.0 NOTE_OFF=57 NOTE_ON=55 TIME_DELTA=8.0 NOTE_OFF=55 BAR_END BAR_START NOTE_ON=55 TIME_DELTA=8.0 NOTE_OFF=55 BAR_END BAR_START NOTE_ON=64 TIME_DELTA=4.0 NOTE_OFF=64 NOTE_ON=62 TIME_DELTA=2.0 NOTE_OFF=62 NOTE_ON=64 TIME_DELTA=2.0 NOTE_OFF=64 NOTE_ON=65 TIME_DELTA=4.0 NOTE_OFF=65 NOTE_ON=64 TIME_DELTA=2.0 NOTE_OFF=64 NOTE_ON=62 TIME_DELTA=2.0 NOTE_OFF=62 BAR_END BAR_START NOTE_ON=60 TIME_DELTA=8.0 NOTE_OFF=60 TIME_DELTA=4.0 NOTE_ON=57 TIME_DELTA=4.0 NOTE_OFF=57 BAR_END INST=4 BAR_START NOTE_ON=49 TIME_DELTA=4.0 NOTE_OFF=49 NOTE_ON=50 TIME_DELTA=2.0 NOTE_OFF=50 NOTE_ON=48 TIME_DELTA=2.0 NOTE_OFF=48 NOTE_ON=47 TIME_DELTA=2.0 NOTE_OFF=47 NOTE_ON=45 TIME_DELTA=2.0 NOTE_OFF=45 NOTE_ON=47 TIME_DELTA=2.0 NOTE_OFF=47 NOTE_ON=43 TIME_DELTA=2.0 NOTE_OFF=43 BAR_END BAR_START NOTE_ON=48 TIME_DELTA=8.0 NOTE_OFF=48 BAR_END BAR_START NOTE_ON=52 TIME_DELTA=4.0 NOTE_OFF=52 NOTE_ON=53 TIME_DELTA=2.0 NOTE_OFF=53 NOTE_ON=52 TIME_

In [ ]:
model = GPT2LMHeadModel.from_pretrained("best_model")

In [ ]:
generate(model, fast_tokenizer, input_data_valid[0])

'INST=3 BAR_START NOTE_ON=57 TIME_DELTA=4.0 NOTE_OFF=57 NOTE_ON=57 TIME_DELTA=4.0 NOTE_OFF=57 NOTE_ON=55 TIME_DELTA=8.0 NOTE_OFF=55 BAR_END BAR_START NOTE_ON=55 TIME_DELTA=8.0 NOTE_OFF=55 BAR_END BAR_START NOTE_ON=64 TIME_DELTA=4.0 NOTE_OFF=64 NOTE_ON=62 TIME_DELTA=2.0 NOTE_OFF=62 NOTE_ON=64 TIME_DELTA=2.0 NOTE_OFF=64 NOTE_ON=65 TIME_DELTA=4.0 NOTE_OFF=65 NOTE_ON=64 TIME_DELTA=2.0 NOTE_OFF=64 NOTE_ON=62 TIME_DELTA=2.0 NOTE_OFF=62 BAR_END BAR_START NOTE_ON=60 TIME_DELTA=8.0 NOTE_OFF=60 TIME_DELTA=4.0 NOTE_ON=57 TIME_DELTA=4.0 NOTE_OFF=57 BAR_END INST=4 BAR_START NOTE_ON=49 TIME_DELTA=4.0 NOTE_OFF=49 NOTE_ON=50 TIME_DELTA=2.0 NOTE_OFF=50 NOTE_ON=48 TIME_DELTA=2.0 NOTE_OFF=48 NOTE_ON=47 TIME_DELTA=2.0 NOTE_OFF=47 NOTE_ON=45 TIME_DELTA=2.0 NOTE_OFF=45 NOTE_ON=47 TIME_DELTA=2.0 NOTE_OFF=47 NOTE_ON=43 TIME_DELTA=2.0 NOTE_OFF=43 BAR_END BAR_START NOTE_ON=48 TIME_DELTA=8.0 NOTE_OFF=48 BAR_END BAR_START NOTE_ON=52 TIME_DELTA=4.0 NOTE_OFF=52 NOTE_ON=53 TIME_DELTA=2.0 NOTE_OFF=53 NOTE_ON=52 TIME_

In [ ]:
generated_sample = []
for sample in input_data_valid:
    generated_sample += [generate(model, fast_tokenizer, sample)]

In [ ]:
with open('output_data.txt', 'w') as f:
    for line in generated_sample:
        f.write(f"{line}\n")

In [ ]:
target_data_valid[0]

'INST=3 BAR_START NOTE_ON=57 TIME_DELTA=4.0 NOTE_OFF=57 NOTE_ON=57 TIME_DELTA=4.0 NOTE_OFF=57 NOTE_ON=55 TIME_DELTA=8.0 NOTE_OFF=55 BAR_END BAR_START NOTE_ON=55 TIME_DELTA=8.0 NOTE_OFF=55 BAR_END BAR_START NOTE_ON=64 TIME_DELTA=4.0 NOTE_OFF=64 NOTE_ON=62 TIME_DELTA=2.0 NOTE_OFF=62 NOTE_ON=64 TIME_DELTA=2.0 NOTE_OFF=64 NOTE_ON=65 TIME_DELTA=4.0 NOTE_OFF=65 NOTE_ON=64 TIME_DELTA=2.0 NOTE_OFF=64 NOTE_ON=62 TIME_DELTA=2.0 NOTE_OFF=62 BAR_END BAR_START NOTE_ON=60 TIME_DELTA=8.0 NOTE_OFF=60 TIME_DELTA=4.0 NOTE_ON=57 TIME_DELTA=4.0 NOTE_OFF=57 BAR_END INST=4 BAR_START NOTE_ON=49 TIME_DELTA=4.0 NOTE_OFF=49 NOTE_ON=50 TIME_DELTA=2.0 NOTE_OFF=50 NOTE_ON=48 TIME_DELTA=2.0 NOTE_OFF=48 NOTE_ON=47 TIME_DELTA=2.0 NOTE_OFF=47 NOTE_ON=45 TIME_DELTA=2.0 NOTE_OFF=45 NOTE_ON=47 TIME_DELTA=2.0 NOTE_OFF=47 NOTE_ON=43 TIME_DELTA=2.0 NOTE_OFF=43 BAR_END BAR_START NOTE_ON=48 TIME_DELTA=8.0 NOTE_OFF=48 BAR_END BAR_START NOTE_ON=52 TIME_DELTA=4.0 NOTE_OFF=52 NOTE_ON=53 TIME_DELTA=2.0 NOTE_OFF=53 NOTE_ON=52 TIME_